# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df_cities = pd.read_csv("../WeatherPy/output.csv")
df_cities


,City No.,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,atuona,-9.8000,-139.0333,77.67,74.0,16.0,14.72,PF,1.635150e+09
1,1,miles city,46.4083,-105.8406,47.52,76.0,1.0,16.11,US,1.635150e+09
2,2,vaitupu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,kampot,10.6167,104.1833,83.17,76.0,98.0,10.04,KH,1.635150e+09
4,4,kuche,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
692,692,alyangula,-13.8483,136.4192,87.67,62.0,59.0,10.36,AU,1.635151e+09
693,693,fairbanks,64.8378,-147.7164,24.76,90.0,75.0,0.00,US,1.635151e+09
694,694,kingaroy,-26.5333,151.8333,64.44,100.0,75.0,8.05,AU,1.635151e+09
695,695,san quintin,30.4833,-115.9500,61.56,89.0,31.0,5.44,MX,1.635151e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [4]:
#defining variable for latitude and longitudes
lat_lng = df_cities[["Lat","Lng"]].dropna()

lat_lng

,Lat,Lng
0,-9.8000,-139.0333
1,46.4083,-105.8406
3,10.6167,104.1833
5,65.6118,72.6842
6,-33.6500,115.3333
...,...,...
692,-13.8483,136.4192
693,64.8378,-147.7164
694,-26.5333,151.8333
695,30.4833,-115.9500


In [5]:
#defining variable for humidity

# Fill NaN values and convert to float
humidity = df_cities["Humidity"].dropna()

humidity

0       74.0
1       76.0
3       76.0
5       79.0
6       52.0
       ...  
692     62.0
693     90.0
694    100.0
695     89.0
696     91.0
Name: Humidity, Length: 635, dtype: float64

In [6]:
# Create heat layer
heat_layer = gmaps.heatmap_layer(lat_lng, weights=humidity, 
                                 dissipating=False, max_intensity=max(humidity),
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure with humidity layer on
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
df_cities["Max Temp"]

0      77.67
1      47.52
2        NaN
3      83.17
4        NaN
       ...  
692    87.67
693    24.76
694    64.44
695    61.56
696    75.18
Name: Max Temp, Length: 697, dtype: float64

In [8]:
df_ideal_cities = df_cities.loc[(df_cities["Max Temp"] > 70) & 
                                (df_cities["Max Temp"] < 80) &
                                (df_cities["Wind Speed"] < 10) &
                                (df_cities["Cloudiness"] == 0), :]

df_ideal_cities

,City No.,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
73,73,cape town,-33.9258,18.4232,71.11,46.0,0.0,5.75,ZA,1.635150e+09
209,209,gat,31.6100,34.7642,78.78,25.0,0.0,6.44,IL,1.635151e+09
282,282,san patricio,28.0170,-97.5169,79.90,95.0,0.0,5.46,US,1.635151e+09
372,372,abha,18.2164,42.5053,76.82,9.0,0.0,8.05,SA,1.635151e+09
458,458,coffs harbour,-30.3000,153.1333,71.87,87.0,0.0,8.05,AU,1.635151e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = df_ideal_cities

hotel_df["Hotel Name"] = ""

/Users/shrutihindocha/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
#searching for hotels within 5000m

base_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    parameters_hotels = {
        "location" : f"{lat},{lng}",
        "keyword" : "Hotel",
        "radius" : 5000,
        "key" : g_key
    }
    
    try:
        response_hotel = requests.get(base_url, params = parameters_hotels).json()

        hotel_df.loc[index,"Hotel Name"] = response_hotel["results"][0]["name"]
    except:
        hotel_df.loc[index,"Hotel Name"] = "Info not found"
        pass

/Users/shrutihindocha/anaconda3/envs/PythonData/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [11]:
hotel_df

,City No.,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
73,73,cape town,-33.9258,18.4232,71.11,46.0,0.0,5.75,ZA,1.635150e+09,The Bay Hotel
209,209,gat,31.6100,34.7642,78.78,25.0,0.0,6.44,IL,1.635151e+09,Desert Gat
282,282,san patricio,28.0170,-97.5169,79.90,95.0,0.0,5.46,US,1.635151e+09,Americas Best Value Inn Sinton
372,372,abha,18.2164,42.5053,76.82,9.0,0.0,8.05,SA,1.635151e+09,Shafa Abha Hotel
458,458,coffs harbour,-30.3000,153.1333,71.87,87.0,0.0,8.05,AU,1.635151e+09,BreakFree Aanuka Beach Resort


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…